## Imports

In [53]:
import logging
import os
from typing import List, Optional

import hydra
import omegaconf
import pytorch_lightning as pl
import torch
from lightning.pytorch import Callback
from omegaconf import DictConfig, ListConfig

from nn_core.callbacks import NNTemplateCore
from nn_core.common import PROJECT_ROOT
from nn_core.common.utils import enforce_tags, seed_index_everything
from nn_core.model_logging import NNLogger
from nn_core.serialization import NNCheckpointIO

# Force the execution of __init__.py if this file is executed directly.
import tvp  # noqa
from tvp.data.datamodule import MetaData
from tvp.data.datasets.registry import get_dataset
from tvp.task_vectors.task_vectors import TaskVector
from tvp.utils.io_utils import load_model_from_artifact
from tvp.utils.utils import build_callbacks
from torch.nn.utils import vector_to_parameters
from torch.nn.utils import parameters_to_vector

pylogger = logging.getLogger(__name__)

torch.set_float32_matmul_precision("high")

## Configuration

In [54]:
%load_ext autoreload
%autoreload 2

import hydra
from hydra import initialize, compose
from typing import Dict, List

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path=str("../conf"), job_name="playground")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


hydra.initialize()

In [55]:
cfg = compose(config_name="task_vectors", overrides=[])

In [56]:
seed_index_everything(cfg)

cfg.core.tags = enforce_tags(cfg.core.get("tags", None))

template_core: NNTemplateCore = NNTemplateCore(
    restore_cfg=cfg.train.get("restore", None),
)
logger: NNLogger = NNLogger(logging_cfg=cfg.train.logging, cfg=cfg, resume_id=template_core.resume_id)

Global seed set to 1608637542


2024-05-18 19:43:47 INFO     Setting seed 1608637542 from seeds[0]                         ]8;id=323618;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/common/utils.py\nn_core.common.utils]8;;\:]8;id=918782;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/common/utils.py#107\107]8;;\

                    INFO     Tags: ['dev']                                                  ]8;id=30561;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/common/utils.py\nn_core.common.utils]8;;\:]8;id=398255;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/common/utils.py#96\96]8;;\

                    INFO     Restoring with mode: <None>                                         ]8;id=852900;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/resume.py\nn_core.resume]8;;\:]8;id=721902;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/resume.py#122\122]8;;\

                    INFO     Instantiating <WandbLogger>                                   ]8;id=130475;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/model_logging.py\nn_core.model_logging]8;;\:]8;id=464412;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/model_logging.py#41\41]8;;\

/home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


## Load models

In [57]:
import copy


zeroshot_identifier = f"{cfg.nn.module.model.model_name}_pt"

zeroshot_model = load_model_from_artifact(artifact_path=f"{zeroshot_identifier}:latest", run=logger.experiment)

finetuned_id_fn = lambda dataset: f"{cfg.nn.module.model.model_name}_{dataset}_{cfg.seed_index}:latest"

finetuned_models = {
    dataset: load_model_from_artifact(artifact_path=finetuned_id_fn(dataset), run=logger.experiment)
    for dataset in cfg.task_vectors.to_apply
}

zeroshot_orig_weights = copy.deepcopy(zeroshot_model.state_dict())

                    INFO     Loading model from artifact ViT-B-16_pt:latest                   ]8;id=224430;file:///media/donato/Extra-storage/Code/model-merging/task-vectors-playground/src/tvp/utils/io_utils.py\tvp.utils.io_utils]8;;\:]8;id=170108;file:///media/donato/Extra-storage/Code/model-merging/task-vectors-playground/src/tvp/utils/io_utils.py#15\15]8;;\

wandb: Downloading large artifact ViT-B-16_pt:latest, 426.51MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.7


Loading ViT-B-16 pre-trained weights.


2024-05-18 19:43:50 INFO     Loading pretrained ViT-B-16 from OpenAI.                                       ]8;id=6511;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/open_clip/factory.py\root]8;;\:]8;id=500215;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

2024-05-18 19:43:52 INFO     Loading model from artifact ViT-B-16_MNIST_0:latest              ]8;id=813694;file:///media/donato/Extra-storage/Code/model-merging/task-vectors-playground/src/tvp/utils/io_utils.py\tvp.utils.io_utils]8;;\:]8;id=822996;file:///media/donato/Extra-storage/Code/model-merging/task-vectors-playground/src/tvp/utils/io_utils.py#15\15]8;;\

...

2024-05-18 19:43:55 INFO     Loading pretrained ViT-B-16 from OpenAI.                                       ]8;id=459697;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/open_clip/factory.py\root]8;;\:]8;id=850440;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

2024-05-18 19:43:57 INFO     Loading model from artifact ViT-B-16_CIFAR100_0:latest           ]8;id=560796;file:///media/donato/Extra-storage/Code/model-merging/task-vectors-playground/src/tvp/utils/io_utils.py\tvp.utils.io_utils]8;;\:]8;id=355368;file:///media/donato/Extra-storage/Code/model-merging/task-vectors-playground/src/tvp/utils/io_utils.py#15\15]8;;\

wandb: Downloading large artifact ViT-B-16_CIFAR100_0:latest, 426.51MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.7


Loading ViT-B-16 pre-trained weights.


2024-05-18 19:44:00 INFO     Loading pretrained ViT-B-16 from OpenAI.                                       ]8;id=255163;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/open_clip/factory.py\root]8;;\:]8;id=527597;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

## Spherical average

In [58]:
import numpy as np


def inner_product(point, tangent_vector_a, tangent_vector_b):
    return torch.tensordot(
        tangent_vector_a, tangent_vector_b, dims=tangent_vector_a.ndim
    )

def dist(point_a, point_b):
    inner = max(min(inner_product(point_a, point_a, point_b), 1), -1)
    return torch.arccos(inner)

def norm(point, tangent_vector):
    return torch.norm(tangent_vector)

def projection(point, vector):
    return vector - inner_product(point, point, vector) * point

def ell_q(q, p):
    vector = projection(q, p - q)
    distance = dist(q, p)
    epsilon = np.finfo(np.float64).eps
    factor = (distance + epsilon) / (norm(q, vector) + epsilon)
    return factor * vector

def exp_q(point, tangent_vector):
    norm_value = norm(point, tangent_vector)
    return point * torch.cos(norm_value) + tangent_vector * torch.sinc(norm_value / np.pi)


In [59]:
from tqdm import tqdm


def spherical_weighted_average(points, weights, tol=1e-8, max_iter=1000, dim=2):
    """
    Compute the spherical weighted average of points on a sphere with given weights using PyTorch.
    
    Args:
    - points (torch.Tensor): A tensor of shape (n, d+1) representing n points on the d-dimensional sphere S^d.
    - weights (torch.Tensor): A tensor of shape (n,) representing the non-negative weights with sum 1.
    - tol (float): Tolerance for the stopping criterion based on the norm of u.
    - max_iter (int): Maximum number of iterations for the main loop.

    Returns:
    - q (torch.Tensor): The spherical weighted average of the input points.
    """

    print(f'Points shape: {points.shape}')
    print(f'Weights shape: {weights.shape}')

    points = points.cuda()
    weights = weights.cuda()

    norm = torch.norm(points, dim=1)

    points =  torch.nn.functional.normalize(points, p=2, dim=1)

    # (num_points, d+1)
    assert points.shape[-1] == dim+1, f"points.shape = {points.shape}, dim = {dim}"

    # points have shape (num_tasks, num_params)
    # weights have shape (num_tasks,)
    
    with torch.no_grad():
        # Ensure weights sum to 1
        weights = weights / weights.sum()
        
        # Initialization, q has shape (d+1,)
        q = (weights[:, None] * points).sum(dim=0)
        q = q / (torch.norm(q) )

        assert q.shape[0] == dim+1, f"q.shape = {q.shape}, dim = {dim}"

        for _ in tqdm(range(max_iter)):
            # Compute p_i^* for each point

            # (num_points, d+1)
            p_i_stars = torch.stack([ell_q(q, p) for p in points])
            u = (weights[:, None] * (p_i_stars )).sum(dim=0) 
            q = exp_q(q, u)

            q = q / (torch.norm(q) )

            # Check if u is sufficiently small
            if torch.norm(u) < tol:
                break
            
            print(f"Norm: {torch.norm(u)}")


    return q * (norm * weights).sum()

## SLERP

In [60]:
import torch
from typing import Union


def slerp(
    t: Union[float, torch.Tensor],
    v0: torch.Tensor,
    v1: torch.Tensor,
    DOT_THRESHOLD: float = 0.9995,
    eps: float = 1e-8,
):
    """
    Spherical linear interpolation using PyTorch

    Args:
        t (float/torch.Tensor): Float value between 0.0 and 1.0
        v0 (torch.Tensor): Starting vector
        v1 (torch.Tensor): Final vector
        DOT_THRESHOLD (float): Threshold for considering the two vectors as
                               colinear. Not recommended to alter this.
    Returns:
        v2 (torch.Tensor): Interpolation vector between v0 and v1
    """
    # Ensure inputs are tensors
    v0 = torch.tensor(v0, dtype=torch.float32).cuda()
    v1 = torch.tensor(v1, dtype=torch.float32).cuda()

    v0_copy = v0.clone()
    v1_copy = v1.clone()

    # Normalize the vectors to get the directions and angles
    v0 = v0 / (torch.norm(v0) + eps)
    v1 = v1 / (torch.norm(v1) + eps)

    # Dot product with the normalized vectors
    dot = torch.sum(v0 * v1)

    # If absolute value of dot product is almost 1, vectors are ~colinear, so use lerp
    if torch.abs(dot) > DOT_THRESHOLD:
        print("colinear vectors")
        return None 

    # Calculate initial angle between v0 and v1
    theta_0 = torch.acos(dot)
    sin_theta_0 = torch.sin(theta_0)

    # Angle at timestep t
    theta_t = theta_0 * t
    sin_theta_t = torch.sin(theta_t)

    # Finish the slerp algorithm
    s0 = torch.sin(theta_0 - theta_t) / sin_theta_0
    s1 = sin_theta_t / sin_theta_0

    res = s0 * v0_copy + s1 * v1_copy

    return res

## Task vectors

In [61]:
flatten = lambda model: parameters_to_vector(model.parameters()) 

def apply_task_vector(model, task_vector):
    model.load_state_dict({k: v + task_vector[k] for k, v in model.state_dict().items()})

In [62]:
zeroshot_vec = flatten(zeroshot_model)

### Aggregate task vectors

In [63]:
with torch.no_grad():

    task_vectors = torch.stack([flatten(finetuned_models[dataset]) - zeroshot_vec for dataset in cfg.task_vectors.to_apply])

### Standard task vectors

In [64]:
task_vectors_sum = task_vectors.sum(dim=0)

### Spherical

In [65]:
weights = torch.full(size=(len(task_vectors),), fill_value=1/len(task_vectors)).cuda()

task_vector_spherical = spherical_weighted_average(copy.deepcopy(task_vectors), weights, tol=1e-7, max_iter=20, dim=task_vectors.shape[-1]-1)
task_vector_spherical = task_vector_spherical.cpu()

Points shape: torch.Size([2, 111792129])
Weights shape: torch.Size([2])


  0%|          | 0/20 [00:00<?, ?it/s]


In [66]:
task_vectors_slerp_torch = slerp(0.5, task_vectors[0], task_vectors[1]).cpu()
print((task_vector_spherical - task_vectors_slerp_torch).norm())

/tmp/ipykernel_998546/3910402856.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v0 = torch.tensor(v0, dtype=torch.float32).cuda()
/tmp/ipykernel_998546/3910402856.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v1 = torch.tensor(v1, dtype=torch.float32).cuda()


tensor(0.4074)


In [67]:
alpha = 0.8

# multi_task_vector = task_vectors_sum / len(task_vectors) #* alpha 
multi_task_vector = task_vector_spherical 
# multi_task_vector = task_vectors_slerp_torch

In [68]:
delta_model = copy.deepcopy(zeroshot_model) 
vector_to_parameters(multi_task_vector, delta_model.parameters())

In [69]:
task_equipped_model = copy.deepcopy(zeroshot_model)
apply_task_vector(task_equipped_model, delta_model.state_dict())

In [70]:
# Apply the resulting task vector
# image_encoder = task_vector_normalized.apply_to(zeroshot_model, scaling_coef=cfg.task_vectors.scaling_coefficient)

classification_head_identifier = f"{cfg.nn.module.model.model_name}_{cfg.nn.data.dataset.dataset_name}_head"
classification_head = load_model_from_artifact(
    artifact_path=f"{classification_head_identifier}:latest", run=logger.experiment
)

model = hydra.utils.instantiate(
    cfg.nn.module, encoder=task_equipped_model, classifier=classification_head, _recursive_=False
)

2024-05-18 19:44:04 INFO     Loading model from artifact ViT-B-16_CIFAR100_head:latest        ]8;id=491560;file:///media/donato/Extra-storage/Code/model-merging/task-vectors-playground/src/tvp/utils/io_utils.py\tvp.utils.io_utils]8;;\:]8;id=836233;file:///media/donato/Extra-storage/Code/model-merging/task-vectors-playground/src/tvp/utils/io_utils.py#15\15]8;;\

wandb:   1 of 1 files downloaded.  
/home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:197: UserWarning: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
  rank_zero_warn(
/home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:197: UserWarning: Attribute 'classifier' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['classifier'])`.
  rank_zero_warn(


## Load dataset

In [71]:
seed_index_everything(cfg)

dataset = get_dataset(
    cfg.nn.data.train_dataset,
    preprocess_fn=model.encoder.train_preprocess,
    location=cfg.nn.data.data_path,
    batch_size=cfg.nn.data.batch_size.train,
)

callbacks: List[Callback] = build_callbacks(cfg.train.callbacks, template_core)

storage_dir: str = cfg.core.storage_dir

pylogger.info("Instantiating the <Trainer>")
trainer = pl.Trainer(
    default_root_dir=storage_dir,
    plugins=[NNCheckpointIO(jailing_dir=logger.run_dir)],
    logger=False,
    callbacks=callbacks,
    **cfg.train.trainer,
    
)

Global seed set to 1608637542


2024-05-18 19:44:06 INFO     Setting seed 1608637542 from seeds[0]                         ]8;id=323618;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/common/utils.py\nn_core.common.utils]8;;\:]8;id=918782;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/common/utils.py#107\107]8;;\

Files already downloaded and verified
Files already downloaded and verified


2024-05-18 19:44:07 INFO     Adding callback <EarlyStopping>                                    ]8;id=30561;file:///media/donato/Extra-storage/Code/model-merging/task-vectors-playground/src/tvp/utils/utils.py\tvp.utils.utils]8;;\:]8;id=398255;file:///media/donato/Extra-storage/Code/model-merging/task-vectors-playground/src/tvp/utils/utils.py#146\146]8;;\

                    INFO     Adding callback <ModelCheckpoint>                                  ]8;id=852900;file:///media/donato/Extra-storage/Code/model-merging/task-vectors-playground/src/tvp/utils/utils.py\tvp.utils.utils]8;;\:]8;id=721902;file:///media/donato/Extra-storage/Code/model-merging/task-vectors-playground/src/tvp/utils/utils.py#146\146]8;;\

...

                    INFO     IPU available: False, using: 0 IPUs           ]8;id=560796;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py\lightning.pytorch.utilities.rank_zero]8;;\:]8;id=355368;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py#63\63]8;;\

INFO: HPU available: False, using: 0 HPUs


                    INFO     HPU available: False, using: 0 HPUs           ]8;id=255163;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py\lightning.pytorch.utilities.rank_zero]8;;\:]8;id=527597;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py#63\63]8;;\

INFO: `Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


                    INFO     `Trainer(val_check_interval=1.0)` was         ]8;id=491560;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py\lightning.pytorch.utilities.rank_zero]8;;\:]8;id=836233;file:///home/donato/miniconda3/envs/tvp/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py#63\63]8;;\
                             configured so validation will run at the end                                          
                             of the training epoch..                                                               

## Evaluation

In [72]:
# pylogger.info("Evaluating on the training set")
# trainer.test(model=model, dataloaders=dataset.train_loader)

pylogger.info("Evaluating on the test set!")
trainer.test(model=model, dataloaders=dataset.test_loader)

                    INFO     Evaluating on the test set!                                                 ]8;id=349771;file:///tmp/ipykernel_998546/1153275028.py\__main__]8;;\:]8;id=650903;file:///tmp/ipykernel_998546/1153275028.py#4\4]8;;\

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:10<00:00,  3.81it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         acc/test          │    0.6944000124931335     │
│         loss/test         │    1.3371354341506958     │
└───────────────────────────┴───────────────────────────┘

[{'acc/test': 0.6944000124931335, 'loss/test': 1.3371354341506958}]